In [ ]:
from core_components.dispatchers.library import BaseEventDispatcher, SystemDispatcher, InputDispatcher

from engine import Engine
a = Engine()
a.start()

In [ ]:
a.roster.player

In [ ]:
a.roster.entities

In [ ]:
from engine import Engine
from core_components.entities.library import PlayerCharactor
from core_components.tiles.base import TileCoordinate, TileTuple
from core_components.atlas import Atlas
import tcod
import numpy as np

player = PlayerCharactor(color=(0,0,0), name="player")
expected_bits = np.full([80,50], fill_value=True)

game = Engine()
print(game.state.map.active.tiles['graphic'][0:2, 0:2])
game.state.map.active.set_state_bits('visible', expected_bits)
game.state.map.active.set_state_bits('seen', expected_bits)
game.state.map.active.update_state()
print(game.state.map.active.tiles['graphic'][0:2, 0:2])


In [ ]:
game.state.map.active.areas['0'].get_random_location()

In [ ]:
import numpy as np
expected_bits = np.full([80,50], fill_value=True)
a.active.set_state_bits('visible', expected_bits)
a.active.visible

In [ ]:
a.active.set_state_bits('seen', expected_bits)

In [ ]:
print(a.active.tiles['graphic'][0:2, 0:2])
a.active.update_state()
print(a.active.tiles['graphic'][0:2, 0:2])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(game.state.map.active.tiles['blocks_movement'])

game.state.map.active.tiles['blocks_vision'] = False



In [ ]:
game.state.roster.player = player
game.state.roster.player.location = TileCoordinate(TileTuple(([10],[10])))  
move_event = tcod.event.KeyDown(
        sym=tcod.event.KeySym.RIGHT,
        scancode=tcod.event.Scancode.RIGHT,
        mod=tcod.event.Modifier.NONE,
        repeat=False,
    )
game.start()

In [ ]:
import time


print(player.location)
game.state.events.put(move_event)
time.sleep(0.2)
print(player.location)

In [ ]:
game.state.map.active.tiles['graphic']

In [ ]:
from engine import Engine
from core_components.events.library import InputEvent, GameStartEvent, GameOverEvent
import tcod
import threading
import time


GAMESTART = GameStartEvent()
GAMEOVER = GameOverEvent('Game Over')
ESCAPE = tcod.event.KeyDown(sym=27, scancode=1, repeat=False, mod=0)

game = Engine()


In [ ]:
game.start()

In [ ]:
print(game.state.game_over.is_set(), game.state.events.qsize(), game.state.actions.qsize())

In [ ]:
game_event = GAMESTART
game.state.events.put(game_event)
time.sleep(0.2)
print(game.state.game_over.is_set(), game.state.events.qsize(), game.state.actions.qsize())

In [ ]:
game_event = ESCAPE
game.state.events.put(game_event)
time.sleep(0.2)
print(game.state.game_over.is_set(), game.state.events.qsize(), game.state.actions.qsize())

In [ ]:
import threading
import sys

def custom_hook(args):
    print(f"Thread failed: {args.thread.name}")
    print(f"Exception type: {args.exc_type}")
    print(f"Exception value: {args.exc_value}")

# Set the global exception hook
threading.excepthook = custom_hook

def faulty_worker():
    raise RuntimeError("Something went wrong!")

thread = threading.Thread(target=faulty_worker)
thread.start()
thread.join()
# Main thread continues after the hook is called
print("Main thread finished")

In [2]:
#!/usr/bin/env python3
import tcod
from engine import Engine
from core_components.entities.library import PlayerCharactor
from core_components.tiles.base import TileCoordinate, TileTuple
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def main() -> None:
    screen_width = 80
    screen_height = 80

    # tileset = tcod.tileset.load_tilesheet(
    #     "oryx_roguelike_160x40.png", columns=8 , rows=2, charmap=tcod.tileset.CHARMAP_CP437
    # )
    
    tileset = tcod.tileset.Tileset(20,20)
    img = Image.open("player.png")
    img = img.convert("RGBA")
    tileset.set_tile(64, np.array(img))
    img = Image.open("monster_20x20.png")
    img = img.convert("RGBA")
    tileset.set_tile(65, np.array(img))

    game = Engine()    
    game.start()

    expected_bits = np.full([screen_width, screen_height], fill_value=True)
    # game.state.map.active.set_state_bits('visible', expected_bits)
    game.map.set_state_bits('seen', expected_bits)
    
    context = tcod.context.new(columns = screen_width, rows = screen_height, tileset=tileset, title="Jay's Roguelike", vsync=True, sdl_window_flags=tcod.context.SDL_WINDOW_RESIZABLE)
    
    while True:
        # Update Console
        game.map.update_state()

        console = context.new_console(screen_width, screen_height, order="F")
        console.rgb[0:screen_width, 0:screen_height] = game.map.get_state()
        if len(game.roster.live_actors) > 0:
            for actor in game.roster.live_actors:
                console.print(actor.location.x, actor.location.y, actor.symbol, fg=actor.color)
                
        context.present(console)
        console.clear()

        # Update Game State
        for game_event in tcod.event.wait():
            if isinstance(game_event, tcod.event.KeyDown):
                game.state.events.put(game_event)

        if game.state.game_over.is_set():
            context.close()   
            break
        
if __name__ == "__main__":
    main()

RENDER:Created renderer: direct3d11


Exiting game.


In [ ]:
a = np.full((8,8), fill_value=False)
#a[2:4, 2:4] = False
a

In [ ]:
b = np.full((8,8), fill_value=False)
b[2:4, 3:5] = True
b

In [ ]:
(a & b).sum(), (a ^ b).sum(), (a | b).sum()

In [ ]:
c = np.full((8,8), fill_value=False)
c[2:4, 2:4] = True
c

In [ ]:
(b & c).sum(), (c ^ b).sum(), (c | b).sum()

In [ ]:
c | b


In [ ]:
img = Image.open("player.png")
img = img.convert("RGBA")
np.array(img)

tileset.set_tile(0, np.array(img))

In [ ]:
tile = tileset.get_tile(ord(chr(0)))
plt.imshow(tile)

In [ ]:
from engine import Engine
from core_components.entities.library import PlayerCharactor
from core_components.tiles.base import TileCoordinate, TileTuple
player = PlayerCharactor(color=(0,0,0), name="player")
another_player = PlayerCharactor(color=(0,0,0), name="another_player")
game.state.roster.player.location = TileCoordinate(TileTuple(([10],[10])))  
game = Engine()